In [1]:
import stratigraph as sg
from mayavi import mlab
import matplotlib.pyplot as plt
import numpy as np
import h5py
from tqdm import trange
import scipy.io as sio
from PIL import ImageFont
from PIL import ImageDraw 

# set up graphics:
%matplotlib qt
plt.rcParams['svg.fonttype'] = 'none'

## Barrell's original plot

Thanks to Matt Hall for helping with this

In [2]:
im = plt.imread('/Users/zoltan/Dropbox/Chronostratigraphy/barrell_strat_plot_copy.png')
max_time = 1e6
max_elevation = 175
x = np.linspace(4.705, 7.857, 1000)
b = np.sin(x)
c = np.sin(x) - 0.25 * np.cos(8 * x) - 0.05 * np.cos(64 * x)
x = (x - 4.705) * max_time/np.max((x - 4.705))
b = (b + 1.0) * 0.396 * max_elevation + 21
c = (c + 1.0) * 0.396 * max_elevation + 21
plt.figure(figsize=(15,8))
plt.imshow(im, extent = [0, max_time, 0, np.max(c)], aspect = 'auto')
time = x
elevation = c
elevation[0] = 0.17 # this is needed to eliminate erosion at the very beginning
elevation[1] = 0.19

plt.plot(time, elevation, linewidth=2)
plt.plot(time, np.minimum.accumulate(elevation[::-1])[::-1], linewidth=2);
plt.xlabel('time (years)', fontsize=16)
plt.ylabel('elevation (m)', fontsize=16);

### Figure 2: The original Barrell plot

In [4]:
fig, dve_data, duration_thickness_data, ts_labels, strat_tops, strat_top_inds, bound_inds, interval_labels = \
        sg.plot_strat_diagram(elevation, 'm', time, 'years', 0.0001, 
        max_elevation=np.max(elevation), max_time=np.max(time), plotting=True, plot_raw_data=True)

### Animation of the original Barrell plot

In [ ]:
# create animation
for end_time in trange(10, 1001, 10):
    fig, dve_data, duration_thickness_data, ts_labels, strat_tops, strat_top_inds, bound_inds, interval_labels = \
            sg.plot_strat_diagram(elevation[:end_time], 'm', time[:end_time], 'years', 0.0001, 
            max_elevation=np.max(elevation), max_time=np.max(time), plotting=True, plot_raw_data=True)
    fname = 'barrell_fig_'+'%03d.png'%(end_time/10)
    fig.savefig(fname, dpi=300)
    plt.close('all')

## The original Wheeler diagram

### Load manually interpreted lines (from original figure)

In [5]:
import glob
fnames = glob.glob('/Users/zoltan/Dropbox/Chronostratigraphy/wheeler_figure*.tif')
fnames.sort()
fnames

['/Users/zoltan/Dropbox/Chronostratigraphy/wheeler_figure_01.tif',
 '/Users/zoltan/Dropbox/Chronostratigraphy/wheeler_figure_02.tif',
 '/Users/zoltan/Dropbox/Chronostratigraphy/wheeler_figure_03.tif',
 '/Users/zoltan/Dropbox/Chronostratigraphy/wheeler_figure_04.tif',
 '/Users/zoltan/Dropbox/Chronostratigraphy/wheeler_figure_05.tif',
 '/Users/zoltan/Dropbox/Chronostratigraphy/wheeler_figure_06.tif',
 '/Users/zoltan/Dropbox/Chronostratigraphy/wheeler_figure_07.tif',
 '/Users/zoltan/Dropbox/Chronostratigraphy/wheeler_figure_08.tif',
 '/Users/zoltan/Dropbox/Chronostratigraphy/wheeler_figure_09.tif',
 '/Users/zoltan/Dropbox/Chronostratigraphy/wheeler_figure_10.tif',
 '/Users/zoltan/Dropbox/Chronostratigraphy/wheeler_figure_11.tif']

### Track lines in images to get topographic and stratigraphic surfaces

In [18]:
XS = []
YS = []
for fname in fnames[:-1]:
    x_pix, y_pix = sg.read_and_track_line(fname)
    delta_x = 10.0
    delta_y = 10.0
    x = 0.5*delta_x + x_pix*delta_x 
    y = 0.5*delta_y + y_pix*delta_y 
    xs, ys = sg.resample_and_smooth(x, y, 50, 5000)
    XS.append(xs)
    YS.append(ys)

min_x = 0
max_x = 10000
for i in range(len(XS)):
    min_x = max(min_x, min(XS[i]))
    max_x = min(max_x, max(XS[i]))

topo = np.zeros((len(XS), 89))
from scipy import interpolate
for i in range(len(XS)):
    x = XS[i]
    y = YS[i]
    f = interpolate.interp1d(x, y)
    xnew = np.arange(min_x, max_x, 50)
    ynew = f(xnew)
    topo[i, :] = ynew
    
topo = 2000 - topo

# convert to stratigraphy:
strat = np.minimum.accumulate(topo[::-1, :], axis=0)[::-1, :] 

plt.figure()
for i in range(strat.shape[0]):
    plt.plot(xnew, strat[i,:], 'k')
plt.axis('equal');

### Create basement surfaces through time

In [19]:
basement_0 = np.min(topo[1, :]) + topo[0, :] - topo[1, :]
basement_7 = topo[0, :]
total_uplift = basement_7 - basement_0
basement_5 = basement_0 + total_uplift * 0.25 # uplift is sped up between time steps 5 and 6
basement_6 = basement_0 + total_uplift * 0.75
basement = np.zeros((topo.shape[0], topo.shape[1]))
basement[0, :] = basement_0
basement[1, :] = basement_0
basement[2, :] = basement_0
basement[3, :] = basement_0
basement[4, :] = basement_0
basement[5, :] = basement_5
basement[6, :] = basement_6
basement[7, :] = basement_7
basement[8, :] = basement_7
basement[9, :] = basement_7

# QC:
plt.figure()
for i in range(topo.shape[0]):
    plt.plot(xnew, strat[i, :], 'k')
plt.axis('equal');
plt.plot(xnew, basement[0, :], 'r')
plt.plot(xnew, basement[-1, :], 'b');

### Figure 4: Plot topographic surfaces at their original, undeformed position

In [22]:
from matplotlib import colormaps
cmap = colormaps['viridis']

fig = plt.figure(figsize=(15, 10))
for end_time in range(10):
    color = cmap(end_time / (topo.shape[0] - 2))
    t = topo[end_time, :] - basement[-1, :] + basement[end_time, :]
    plt.plot(xnew, t, color=color)
plt.axis('equal');

### Figure 5: Plot stratigraphy at a given time step

In [23]:
for end_time in range(9,10):
    fig = plt.figure(figsize=(15, 10))
    t = np.zeros((end_time+1, topo.shape[1]))
    for i in range(end_time+1):
        t[i, :] = topo[i, :] - basement[-1, :] + basement[end_time, :]
    strat = np.minimum.accumulate(t[::-1, :], axis=0)[::-1, :]
    for i in range(strat.shape[0]-1):
        x = np.hstack((xnew, xnew[::-1]))
        y = np.hstack((strat[i, :], strat[i+1, :][::-1]))
        plt.fill(x, y, facecolor='palegoldenrod', edgecolor='k', alpha=0.5)
    x = np.hstack((xnew[0], xnew[-1], xnew[::-1]))
    y = np.hstack((-275, -275, basement[end_time, :][::-1]))
    plt.fill(x, y, facecolor='lightgray', edgecolor='k', alpha=0.5)
    plt.axis('equal')
    plt.ylim(-654.8220305610621, 2514.597324277648)
    plt.tight_layout()
#     fname = 'wheeler_fig_'+'%03d.png'%(end_time)
#     fig.savefig(fname)

### Resample surfaces to a higher resolution (so that an animation can be built)

In [28]:
time = np.arange(0, 10) * 1e6 # time vector (years)

sampling_rate = 250000 # years
time1, elevation1 = sg.resample_elevation_spl(time, t[:, 7], sampling_rate)
plt.figure()
plt.plot(time, t[:, 7], '.-')
plt.plot(time1, elevation1, '.-')

topo1 = np.zeros((len(time1), topo.shape[1])) # array for resampled topo surfaces
for i in range(topo.shape[1]):
    elevation = t[:, i].copy()
    time1, elevation = sg.resample_elevation_spl(time, elevation, sampling_rate)
    topo1[:, i] = elevation

# this is needed so that there is no erosion at the bottom left side:
topo1[1, :] = np.maximum(topo1[0, :], topo1[1, :])
topo1[2, :] = np.maximum(topo1[0, :], topo1[2, :])
topo1[3, :] = np.maximum(topo1[0, :], topo1[3, :])
        
subsid = np.zeros((len(time1), basement.shape[1])) # subsidence / uplift array
for i in range(basement.shape[1]):
    elevation = basement[:, i].copy()
    time1, elevation = sg.resample_elevation_int1d(time, elevation, sampling_rate)
    subsid[:, i] = elevation
    
plt.figure()
for i in range(topo1.shape[0]):
    plt.plot(xnew, topo1[i, :], 'k')
plt.axis('equal');

### See how the resulting stratigraphy looks like

In [30]:
end_time = 36
fig = plt.figure(figsize=(15, 10))
t = np.zeros((end_time+1, topo1.shape[1]))
for i in range(end_time+1):
    t[i, :] = topo1[i, :] - subsid[-1, :] + subsid[end_time, :]
strat = np.minimum.accumulate(t[::-1, :], axis=0)[::-1, :]
for i in range(strat.shape[0]-1):
    x = np.hstack((xnew, xnew[::-1]))
    y = np.hstack((strat[i, :], strat[i+1, :][::-1]))
    plt.fill(x, y, facecolor='palegoldenrod', edgecolor='k', alpha=0.5)
x = np.hstack((xnew[0], xnew[-1], xnew[::-1]))
y = np.hstack((-275, -275, subsid[end_time, :][::-1]))
plt.fill(x, y, facecolor='lightgray', edgecolor='k', alpha=0.5)
plt.axis('equal')
plt.ylim(-654.8220305610621, 2514.597324277648)
plt.tight_layout()

### Figure 3: Plot stratigraphy and Wheeler diagram

In [31]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

for end_time in range(36, 37):  # change '36' to '0' to plot all time steps
    fig = plt.figure(figsize=(7.5,10))
    ax1 = fig.add_axes([0.1, 0.70, 0.85, 0.25]) # [left, bottom, width, height]
    ax2 = fig.add_axes([0.1, 0.05, 0.85, 0.25*2.4]) # [left, bottom, width, height]
    t = np.zeros((end_time+1, topo1.shape[1]))
    for i in range(end_time+1):
        t[i, :] = topo1[i, :] - subsid[-1, :] + subsid[end_time, :]
    strat = np.minimum.accumulate(t[::-1, :], axis=0)[::-1, :]
    for i in range(strat.shape[0]-1):
        x = np.hstack((xnew, xnew[::-1]))
        y = np.hstack((strat[i, :], strat[i+1, :][::-1]))
        ax1.fill(x, y, facecolor='palegoldenrod', edgecolor=None, linewidth = 0.5)
        if np.mod(i, 4) == 0:
            ax1.plot(xnew, strat[i, :], 'k', linewidth = 0.5)
    if end_time >= 1:
        ax1.plot(xnew, strat[i+1, :], 'k', linewidth = 1)
    x = np.hstack((xnew[0], xnew[-1], xnew[::-1]))
    y = np.hstack((-260, -260, subsid[end_time, :][::-1]))
    ax1.fill(x, y, facecolor='lightgray', edgecolor='k')
    ax1.set_xlim(xnew[0], xnew[-1])
    ax1.set_ylim(-260, 1600)
    ax1.set_xlabel('distance (m)', fontsize=12)
    ax1.set_ylabel('distance (m)', fontsize=12)
    ax1.tick_params(axis='both', labelsize=8)

    strat, wheeler, wheeler_strat, vacuity, stasis = sg.create_wheeler_diagram_2D(topo1.T, 0.1)
    
    im = ax2.imshow(wheeler_strat.T, cmap='RdBu', vmin = -100, vmax = 100, 
               interpolation='none', aspect='auto', extent = [xnew[0], xnew[-1], time1[-1]/1e6, 0])
    plt.gca().invert_yaxis()
    axins1 = inset_axes(ax2,
                        width="100%",  # width = 50% of parent_bbox width
                        height="100%",  # height : 5%
                        loc='upper right',
                        bbox_to_anchor=(0.47, 0.95, 0.5, 0.035),  # x, y, width, height
                        bbox_transform=ax2.transAxes)
    axins1.set_position([0.10, 0.619, 0.94, 0.643])
    axins1.set_xticks([-100, -50, 0, 50, 100])
    axins1.set_xticklabels([-100, -50, 0, 50, 100], fontsize=8)
    fig.colorbar(im, cax=axins1, orientation="horizontal")
    ax2.set_xlabel('distance (m)', fontsize=12)
    ax2.set_ylabel('time (My)', fontsize=12)
    ax2.text(2030, 7.85, 'erosion (m)', fontsize=12, zorder=200)
    ax2.text(3430, 7.85, 'deposition (m)', fontsize=12, zorder=200)
    ax2.tick_params(axis='both', labelsize=8)
    if end_time < 36:
        ax2.fill([xnew[0]+5, xnew[-1]-5, xnew[-1]-5, xnew[0]+5], [time1[-1]/1e6-0.01, time1[-1]/1e6-0.01, 
                                        time1[end_time]/1e6+0.01, time1[end_time]/1e6+0.01], 'w', zorder=100)
#     fname = 'wheeler_fig_'+'%03d.png'%(end_time)
#     fig.savefig(fname, dpi=300)
#     plt.close('all')

### Figure 5: Plot stratigraphy for ten time steps

In [32]:
# create figure with all 10 time steps:
fig, axes = plt.subplots(5, 2, figsize=(10,10))
for end_time in range(10):
    t = np.zeros((end_time+1, topo.shape[1]))
    for i in range(end_time+1):
        t[i, :] = topo[i, :] - basement[-1, :] + basement[end_time, :]
    strat = np.minimum.accumulate(t[::-1, :], axis=0)[::-1, :]
    if end_time < 5:
        ax = axes[end_time, 0]
    else:
        ax = axes[end_time-5, 1]
    for i in range(strat.shape[0]-1):
        x = np.hstack((xnew, xnew[::-1]))
        y = np.hstack((strat[i, :], strat[i+1, :][::-1]))
        ax.fill(x, y, facecolor='palegoldenrod', edgecolor=None, linewidth = 0.5)
        ax.plot(xnew, strat[i, :], 'k', linewidth = 0.5)
    if end_time >= 1:
        ax.plot(xnew, strat[i+1, :], 'k', linewidth = 1)
    x = np.hstack((xnew[0], xnew[-1], xnew[::-1]))
    y = np.hstack((-260, -260, basement[end_time, :][::-1]))
    ax.fill(x, y, facecolor='lightgray', edgecolor='k')
    ax.set_xlim(xnew[0], xnew[-1])
    ax.set_ylim(-260, 1600)
    ax.tick_params(axis='both', labelsize=8)